In [5]:
import sys
!{sys.executable} -m pip install --user findspark

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType, ArrayType, LongType, DoubleType

In [11]:
from scorers import score_text
from spark_tools import SparkSentimentStreamer

In [10]:
sc = SparkContext("local[*]", "NetworkWordCount")
ssc = StreamingContext(sc, 1)
spark = SparkSession \
    .builder \
    .appName("SentimentWikiProcessor") \
    .getOrCreate()

In [13]:
dataInp = "streamInput/"
dataOut = "streamOut/"

In [14]:
streamer = SparkSentimentStreamer(sc, ssc, spark, score_text, dataInp, dataOut)

In [ ]:
streamer.run()

In [16]:
streamer.stop()